# Getting Started With Text Embeddings
### 1. Vertex AI by google cloud 
### 2. OpenAI

## 1.  Vertex AI (google cloud)

Check out the tutorial by Andrew Ng: https://learn.deeplearning.ai/courses/google-cloud-vertex-ai/lesson/2/getting-started-with-text-embeddings

#### Project environment setup (Go to the course website so you don't need to setup on your own)

- Load credentials and relevant Python Libraries
- If you were running this notebook locally, you would first install Vertex AI.  In this classroom, this is already installed.
```Python
!pip install google-cloud-aiplatform
```


In [1]:
from utils import authenticate
credentials, PROJECT_ID = authenticate() # Get credentials and project ID

#### Enter project details

In [2]:
print(PROJECT_ID)

DLAI_PROJECT


In [3]:
REGION = 'us-central1'

In [4]:
# Import and initialize the Vertex AI Python SDK

import vertexai
vertexai.init(project = PROJECT_ID, 
              location = REGION, 
              credentials = credentials)

#### Use the embeddings model
- Import and load the model.

In [5]:
from vertexai.language_models import TextEmbeddingModel

In [6]:
embedding_model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@001")

- Generate a word embedding

In [7]:
embedding = embedding_model.get_embeddings(
    ["life"])

- The returned object is a list with a single `TextEmbedding` object.
- The `TextEmbedding.values` field stores the embeddings in a Python list.

In [8]:
vector = embedding[0].values
print(f"Length = {len(vector)}")
print(vector[:10])

Length = 768
[-0.006005102302879095, 0.015532972291111946, -0.030447669327259064, 0.05322219058871269, 0.014444807544350624, -0.0542873740196228, 0.045140113681554794, 0.02127358317375183, -0.06537645310163498, 0.019103270024061203]


- Generate a sentence embedding.

In [9]:
embedding = embedding_model.get_embeddings(
    ["What is the meaning of life?"])

In [10]:
vector = embedding[0].values
print(f"Length = {len(vector)}")
print(vector[:10])

Length = 768
[0.020522113889455795, 0.02229207195341587, -0.009265718050301075, 0.005001612473279238, 0.016248879954218864, -0.018983161076903343, 0.04320966452360153, 0.02643178217113018, -0.04369377717375755, 0.023666976019740105]


#### Similarity

- Calculate the similarity between two sentences as a number between 0 and 1.
- Try out your own sentences and check if the similarity calculations match your intuition.

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
emb_1 = embedding_model.get_embeddings(
    ["What is the meaning of life?"]) # 42!

emb_2 = embedding_model.get_embeddings(
    ["How does one spend their time well on Earth?"])

emb_3 = embedding_model.get_embeddings(
    ["Would you like a salad?"])

vec_1 = [emb_1[0].values]
vec_2 = [emb_2[0].values]
vec_3 = [emb_3[0].values]

- Note: the reason we wrap the embeddings (a Python list) in another list is because the `cosine_similarity` function expects either a 2D numpy array or a list of lists.
```Python
vec_1 = [emb_1[0].values]
```

In [13]:
print(cosine_similarity(vec_1,vec_2)) 
print(cosine_similarity(vec_2,vec_3))
print(cosine_similarity(vec_1,vec_3))

[[0.65503744]]
[[0.52001556]]
[[0.54139322]]


#### From word to sentence embeddings
- One possible way to calculate sentence embeddings from word embeddings is to take the average of the word embeddings.
- This ignores word order and context, so two sentences with different meanings, but the same set of words will end up with the same sentence embedding.

In [14]:
in_1 = "The kids play in the park."
in_2 = "The play was for kids in the park."

- Remove stop words like ["the", "in", "for", "an", "is"] and punctuation.

In [15]:
in_pp_1 = ["kids", "play", "park"]
in_pp_2 = ["play", "kids", "park"]

- Generate one embedding for each word.  So this is a list of three lists.

In [16]:
embeddings_1 = [emb.values for emb in embedding_model.get_embeddings(in_pp_1)]

- Use numpy to convert this list of lists into a 2D array of 3 rows and 768 columns.

In [17]:
import numpy as np
emb_array_1 = np.stack(embeddings_1)
print(emb_array_1.shape)

(3, 768)


In [18]:
embeddings_2 = [emb.values for emb in embedding_model.get_embeddings(in_pp_2)]
emb_array_2 = np.stack(embeddings_2)
print(emb_array_2.shape)

(3, 768)


- Take the average embedding across the 3 word embeddings 
- You'll get a single embedding of length 768.

In [19]:
emb_1_mean = emb_array_1.mean(axis = 0) 
print(emb_1_mean.shape)

(768,)


In [20]:
emb_2_mean = emb_array_2.mean(axis = 0)

- Check to see that taking an average of word embeddings results in two sentence embeddings that are identical.

In [21]:
print(emb_1_mean[:4])
print(emb_2_mean[:4])

[-0.00385805 -0.00522636  0.00574341  0.03331106]
[-0.00385805 -0.00522636  0.00574341  0.03331106]


#### Get sentence embeddings from the model.
- These sentence embeddings account for word order and context.
- Verify that the sentence embeddings are not the same.

In [22]:
print(in_1)
print(in_2)

The kids play in the park.
The play was for kids in the park.


In [23]:
embedding_1 = embedding_model.get_embeddings([in_1])
embedding_2 = embedding_model.get_embeddings([in_2])

In [24]:
vector_1 = embedding_1[0].values
print(vector_1[:4])
vector_2 = embedding_2[0].values
print(vector_2[:4])

[0.0039385221898555756, -0.020830577239394188, -0.002994248876348138, -0.007580515928566456]
[-0.01565515622496605, -0.012884826399385929, 0.01229254249483347, -0.0005865463172085583]


## 2. OpenAI

#### Project environment setup

In [1]:
from openai import AzureOpenAI

In [2]:
# Parameters
client = AzureOpenAI(
  azure_endpoint = "https://hkust.azure-api.net",
  api_version = "2023-05-15",
  api_key = "<your openai api key>" #put your api key here
)


In [3]:
word_embedding = client.embeddings.create(
        input=["life"],
        model="text-embedding-ada-002", # The only embedding model available for ust
        # dimensions=... # You could limit the number of output dimensions with the new embeddings models
    )

In [4]:
print(word_embedding)

CreateEmbeddingResponse(data=[Embedding(embedding=[0.016394486650824547, -0.01343799103051424, 0.0005860686069354415, -0.028179310262203217, -0.0034075151197612286, 0.004036885220557451, -0.009864130057394505, -0.0011301219929009676, -0.01098224800080061, -0.011057703755795956, 0.0063348570838570595, 0.025970513001084328, 0.014816774055361748, 0.0037590705323964357, 0.0032325948122888803, 0.008416750468313694, 0.05334038659930229, -0.009377097710967064, 0.03416089713573456, -0.03018231876194477, -0.016806064173579216, 0.02004380337893963, -4.7427824029000476e-05, -0.020180994644761086, -0.006441181059926748, 0.001076102489605546, 0.013952462002635002, -0.01698441430926323, 0.015461578033864498, -0.01635332778096199, 0.035971835255622864, -0.02845369465649128, -0.02366568148136139, -0.00697966106235981, 0.010296286083757877, -0.020716045051813126, -0.02019471488893032, -0.011675069108605385, -0.0031982967630028725, 0.0010683854343369603, 0.007339790929108858, 0.011942594312131405, -0.01

In [7]:
len(word_embedding.data[0].embedding)

1536

In [8]:
# Define a helper function to calculate embeddings
def get_embedding_vec(input):
  """Returns the embeddings vector for a given input"""
  return client.embeddings.create(
        input=input,
        model="text-embedding-ada-002", # The only embedding model available for ust
        # dimensions=... # You could limit the number of output dimensions with the new embeddings models
    ).data[0].embedding

#### Use the embeddings model

In [9]:
emb_1_ada = get_embedding_vec(
    ["What is the meaning of life?"]) 


In [10]:
emb_2_ada = get_embedding_vec(
    ["How does one spend their time well on Earth?"])

emb_3_ada = get_embedding_vec(
    ["Would you like a salad?"])

In [16]:
vec_1_ada = [emb_1_ada]
vec_2_ada = [emb_2_ada]
vec_3_ada = [emb_3_ada]

In [17]:
print(cosine_similarity(vec_1_ada,vec_2_ada)) 
print(cosine_similarity(vec_2_ada,vec_3_ada))
print(cosine_similarity(vec_1_ada,vec_3_ada))

[[0.8298554]]
[[0.75493789]]
[[0.76047052]]
